# Growth Data Analyst Exercise: Full report
#### By Umar Butt, Friday 9-NOV-2024

**Summary**

This `analysis` focused on evaluating verification success rates `across three test groups` using `SMS` and `WhatsApp` as verification `methods`. `Group A (SMS-only)` demonstrated the lowest success rate (0.8735), while `Groups B (SMS > WhatsApp)`and `C (WhatsApp > SMS)`, which offered both methods, achieved higher rates, suggesting that providing users with multiple verification options enhances success. Group C had the highest overall success rate at 0.9282, with SMS achieving the highest individual success rate within this group (0.9535).

The findings show that SMS generally `outperforms` WhatsApp in verification success, particularly when offered as a secondary option. User preference tends to align with the first method offered, as shown by the higher WhatsApp uptake in Group C and higher SMS uptake in Group B. `A key insight `is that a two-method setup increases the likelihood of successful verification compared to a single-method approach.

Further analysis could integrate cost data by region, user demographics, and connectivity to optimise method selection for different user segments. Overall, incorporating a dual-method approach while prioritising SMS where feasible appears to be a reliable strategy for improving verification success.

**Note on Data Integrity:** Some rows were removed due to either null values or a lack of reference in another table. This data cleanup process is detailed in the `00_data_cleanup_and_starschema_constraints.sql` file, ensuring more accurate analysis by excluding incomplete data points.

**References:**
- Additional to this report file name:  
  `02_report_growth_data_analyst_exercise`  
  For more information and access to the project, visit [GitHub repository](https://github.com/chemi5t/chemi5t-muzz-growth-data-analyst).

**Mini-reports have been carried out for insights:**
- `03_report_verification_and_cost_analysis`
- `04_age_bracket_method_failure_rates`

---

**1. Which screen would you suggest we proceed with?**  
To determine the optimal screen, verification success rates were analysed across methods and groups, aiming to select the screen configuration with the highest success rate for future use.

**1.a. Verification success rates by group and method:**  
This query identifies the combination of group and method with the highest verification success rate.


In [1]:
import pandas as pd
from database_utils import DatabaseConnector as dc
from decouple import config

# Initialise the DatabaseConnector instance
db_connector = dc()

# Load the database credentials (assuming 'config' pulls from your .env or credentials file)
db_credentials = db_connector.read_db_creds(config('credentials_env'))  # Update if needed

# Initialize the database engine
engine = db_connector.init_db_engine(db_credentials)

# Check if the engine is initialized
if engine is None:
    print("Failed to initialize the database engine.")
else:
    # Now you can execute your SQL query and load it into a Pandas DataFrame
    query_1a = '''
    SELECT 
        v.group, 
        v.method, 
        ROUND(AVG(v.verified), 4) AS verification_success_rate
    FROM 
        fact_verification AS v
    GROUP BY 
        v.group, v.method
    ORDER BY 
        verification_success_rate DESC;
    '''
    
    # Use pd.read_sql_query to directly execute the query and return the result as a DataFrame
    result_df = pd.read_sql_query(query_1a, engine)

    # Display the DataFrame (first few rows)
    display(result_df.head())


,group,method,verification_success_rate
0,C,Sms,0.9535
1,B,Sms,0.9298
2,B,Whatsapp,0.9144
3,C,Whatsapp,0.9140
4,A,Sms,0.8735


**1.b. Verification success rates by group:**

This query calculates the success rate by group and the representation percentage of each group in the overall data set.



In [2]:
# Define the new SQL query
query_1b = '''
SELECT 
    v.group, 
    ROUND(AVG(v.verified), 4) AS verification_success_rate,
    ROUND(COUNT(*) * 100 / SUM(COUNT(*)) OVER (), 2) AS group_representation_percentage
FROM 
    fact_verification AS v
GROUP BY 
    v.group
ORDER BY 
    verification_success_rate DESC;
'''

# Execute the new query and load the result into a DataFrame
result_df2 = pd.read_sql_query(query_1b, engine)

# Display the new DataFrame (first few rows)
display(result_df2.head())


,group,verification_success_rate,group_representation_percentage
0,C,0.9282,33.21
1,B,0.9281,32.16
2,A,0.8735,34.63


**Insights:**

- **Highest Success Rate:** 
    - Group C (WhatsApp > SMS) `demonstrates the highest success rate` (0.9282), followed closely by Group B (SMS > WhatsApp) at 0.9281.
- **Lowest Success Rate:** 
    - Group A (SMS-only) has the `lowest success rate` of 0.8735, indicating that including an alternative verification method may enhance effectiveness.
- **Representation:** 
    - `Distribution` among test groups is fairly `balanced`: Group A makes up 34.64% of total participants, Group B 32.16%, and Group C 33.21%.

---

**2. Which success metrics did you consider and why?**

To evaluate each method’s performance, I analysed success rates by `method` within each `group`, especially for Groups B and C, where users had access to both SMS and WhatsApp. This query retrieves essential metrics, including `success rate` and `usage distribution`.

In [7]:
query_2 = '''
SELECT 
    v.group, 
    v.method, 
    ROUND(AVG(v.verified), 4) AS verification_success_rate,
    ROUND(COUNT(*) / SUM(COUNT(*)) OVER (), 4) AS method_representation_percentage_within_group,
    ROUND(COUNT(*) * 100.0 / SUM(COUNT(*)) OVER (PARTITION BY v.group), 2) AS method_share_within_group_percent
FROM 
    fact_verification AS v
GROUP BY 
    v.group, v.method
ORDER BY 
    verification_success_rate DESC;
'''

# Execute the query and store results in a DataFrame
result_df3 = pd.read_sql_query(query_2, engine)

# Display the DataFrame with the new query result
display(result_df3.head())

,group,method,verification_success_rate,method_representation_percentage_within_group,method_share_within_group_percent
0,C,Sms,0.9535,0.1201,36.17
1,B,Sms,0.9298,0.2854,88.76
2,B,Whatsapp,0.9144,0.0362,11.24
3,C,Whatsapp,0.9140,0.2120,63.83
4,A,Sms,0.8735,0.3463,100.00


**Insights:**

- **Group A (SMS-only):** 
    - Success rate of 0.8735, with SMS used exclusively.
- **Group B (SMS > WhatsApp):** 
    - SMS has a 0.9298 success rate, representing 88.76% of group interactions. WhatsApp has a slightly lower success rate of 0.9144, representing 11.24% of the group.
- **Group C (WhatsApp > SMS):** 
    - `WhatsApp` is used more `frequently` (63.83%) but has a `lower success rate` (0.9140). `SMS`, though less used (36.17%), has the `highest success rate across all methods` at 0.9535.

**Key Observations:**

- **Dual-Method Effectiveness:** 
    - Groups B and C, both of which offer SMS and WhatsApp, show higher success rates than Group A, suggesting that `providing a choice` of methods may increase verification success.
- **Higher Success with SMS:** 
    - Within Groups B and C, SMS `consistently outperforms` WhatsApp, especially in Group C, where SMS achieves a 0.9535 success rate, indicating it may be a `more reliable option` in some cases.
- **First Method Preference:** 
    - In groups with two methods, the `first method is often preferred`: Group C (WhatsApp > SMS) favors WhatsApp at 63.83%, while Group B (SMS > WhatsApp) heavily favors SMS at 88.74%. Group C’s lower performance when WhatsApp is prioritised suggests that SMS could be a better first choice.

---

**3. Would you incorporate additional data if you could?**

- **Additional Data:** 
    - Including data on method costs and regional distribution would offer valuable insights into the cost-effectiveness and performance of verification methods. This would enable refined recommendations on method allocation based on geographic differences and regional preferences.


In [8]:
query_3 = '''
SELECT 
    u.country, 
    v.method,
    COUNT(v."userID") AS user_count,
    ROUND(
        CASE 
            WHEN v.method = 'Whatsapp' THEN COUNT(v."userID") * c.whatsapp_usd::numeric
            WHEN v.method = 'Sms' THEN COUNT(v."userID") * c.sms_usd::numeric
        END, 
        2
    ) AS total_cost,
    ROUND(AVG(v.verified), 4) AS verification_success_rate
FROM 
    fact_verification AS v
JOIN 
    dim_user AS u 
ON 
    v."userID" = u."userID"
JOIN 
    dim_country AS c 
ON 
    u.country = c.country
GROUP BY 
    u.country, v.method, c.whatsapp_usd, c.sms_usd
ORDER BY 
    total_cost DESC;
'''

# Execute the query and store results in a DataFrame
result_df3 = pd.read_sql_query(query_3, engine)

# Display the DataFrame with the new query result
display(result_df3)


,country,method,user_count,total_cost,verification_success_rate
0,PK,Sms,1096,197.28,0.8823
1,MA,Sms,1511,181.32,0.8776
2,ID,Sms,711,149.48,0.8650
3,EG,Sms,804,102.98,0.8682
4,DZ,Sms,539,70.07,0.8998
...,...,...,...,...,...
265,RE,Whatsapp,1,0.04,0.0000
266,TD,Whatsapp,1,0.04,1.0000
267,BO,Whatsapp,1,0.04,1.0000
268,TG,Whatsapp,1,0.04,1.0000


**Explanation of Results:**

The query is designed to calculate the total cost of each verification method (SMS and WhatsApp) per country, based on the number of users and the unit costs (`whatsapp_usd` and `sms_usd`). It also computes the verification success rate for each method by country. This allows for a cost-performance evaluation, which could influence decision-making on which verification method to prioritize in each region.

---

**4. Why do you think your chosen screen performed best?**

The chosen screen, which offers both WhatsApp and SMS verification options (Group C: WhatsApp > SMS), performed best due to a few key factors:

- **Increased Flexibility**: Offering two methods gives users the flexibility to select their preferred verification method, which may boost comfort and ease of use, enhancing success rates.

- **Higher Success Rates with SMS as a Secondary Option**: While WhatsApp was the primary option in this group, SMS verification proved to have the highest individual success rate across all groups (0.9535). This suggests that users are more likely to succeed when SMS is available as a fallback option, especially for those who may face connectivity issues or limitations with WhatsApp.

- **User Preference Alignment**: Data suggests that when presented with multiple options, users tend to select the first method offered, as shown by the higher WhatsApp uptake in Group C. This initial engagement likely contributes to the group’s high overall success rate (0.9282).

Overall, the dual-method setup, paired with prioritising WhatsApp but retaining SMS as a secondary option, likely optimises both user satisfaction and verification success.

---

**5. Would you recommend any additional changes to the screen?**

Yes, I would recommend the following adjustments to further enhance the screen's effectiveness based on observed success rates and user preferences:

- **Highlight SMS as a Recommended Option**: Given that SMS generally has a higher individual success rate, especially as a secondary choice (0.9535 in Group C), `subtly recommending it` could improve overall verification rates. This can be achieved without overwhelming users by framing SMS as a reliable fallback option.

- **Implement a Hybrid Model**: Allow `users to select their preferred` primary method, with an option to switch if the initial method fails. This flexibility may improve verification success, particularly for users in regions with connectivity challenges that impact WhatsApp or SMS availability.

- **Contextual Guidance**: SMS could be recommended in regions with low internet connectivity, guiding users toward the method most likely to succeed in their situation.

- **Regular Evaluation and Adjustment**: Conduct `periodic evaluations` of verification success rates by region and adjust the method allocation `based on regional performance`, user preference, and cost efficiency. This could involve making SMS the default method in areas where it outperforms WhatsApp and prioritising WhatsApp in regions where it is more successful.

- **Incorporate User Feedback Mechanisms**: Including a `feedback option` for users who experience issues with their chosen method can help refine the system for future users. Analysing this feedback would also allow for further personalisation and `regional adjustments` in verification processes.

---

**Summary of Findings and Recommendations**

This analysis supports the effectiveness of a two-method setup, particularly when SMS is prioritised or recommended as a fallback. The recommendations aim to enhance the user experience while maximising verification success across diverse user segments. Future improvements should integrate regional performance, user preference, and cost data to ensure the verification process remains efficient, cost-effective, and user-friendly.
